## Comprovació absència NA i Outliers df

In [2]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col

sc = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/12 11:03:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
paths = ['data_lake/compravenda_sup.parquet', 'data_lake/rent_price.parquet', 'data_lake/renda.parquet']
col_numeriques = {'data_lake/compravenda_sup.parquet': ['Nombre']
                  , 'data_lake/renda.parquet': ['Import_Euros']
                , 'data_lake/rent_price.parquet': ['Price']}

In [11]:
for path in paths:
    RDD = sc.read.parquet(path)
    for columna in col_numeriques[path]:
        valor_min = RDD.rdd.filter(lambda f: '-' not in str(f[columna]))\
            .map(lambda x: (1, x[columna]))\
            .mapValues(lambda x: float(x) if type(x) == str else x)\
            .reduceByKey(lambda x, y: min(x,y)).collect()[0][1]
        valor_max = RDD.rdd.filter(lambda f: '-' not in str(f[columna]))\
            .map(lambda x: (1, float(x[columna])))\
            .reduceByKey(lambda x, y: max(x,y)).collect()[0][1]
        
        RDD = RDD.withColumn(columna, (RDD[columna] - valor_min)/ (valor_max - valor_min))

    # Verificar si hay valores NA
    na_count = RDD.na.drop().count()
    if na_count == RDD.count():
        print(f"No hay valores NA en el DataFrame {path}")
    else:
        print(f"Hay valores NA en el DataFrame {path}")

    RDD.distinct().show()
    print(f"El número de filas distintas en el DataFrame {path} es {RDD.distinct().count()} de un total de {RDD.count()} filas")

    # falta el RDD.write

Hay valores NA en el DataFrame data_lake/compravenda_sup.parquet
+----+---------+--------------+-------------------+----------+--------------------+----------------------------------+-------------------+
| Any|Trimestre|Codi_Districte|      Nom_Districte|Codi_Barri|           Nom_Barri|Superfície_mitjana_(m2_construïts)|             Nombre|
+----+---------+--------------+-------------------+----------+--------------------+----------------------------------+-------------------+
|2023|        1|             2|           Eixample|         7|la Dreta de l'Eix...|                             Total|               0.22|
|2023|        1|            10|         Sant Martí|        72|Sant Martí de Pro...|                    Habitatge usat|0.11129411764705882|
|2023|        2|             3|     Sants-Montjuïc|        12|la Marina del Pra...|                             Total|0.15764705882352942|
|2023|        2|             8|         Nou Barris|        46| el Turó de la Peira|                  

In [7]:
for path in paths:
    RDD = sc.read.parquet(path)
    for columna in col_numeriques[path]:
        RDD.approxQuantile(columna, [0.25, 0.5, 0.75], 0)

IllegalArgumentException: requirement failed: Quantile calculation for column Nombre with data type StringType is not supported.

In [5]:
for path in paths:
    RDD = sc.read.parquet(path)
    for columna in col_numeriques[path]:
        IQRdf = RDD.groupby(columna)\
            .agg(F.expr('percentile(columna, array(0.25))')[0].alias('lower_quartile'), F.expr('percentile(columna, array(0.75))')[0].alias('upper_quartile'), F.expr('percentile(columna, array(0.5))')[0].alias('duration_median'))\
            .withColumn("quartile_deviation", (F.col("upper_quartile") - F.col("lower_quartile"))/2)

outliersremoved = RDD.join(IQRdf, on=columna, how='inner')\
    .where(col(columna) > col("lower_quartile") - 1.5*col("quartile_deviation")) & (col(columna) < col("upper_quartile") + 1.5*col("quartile_deviation"))

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `columna` cannot be resolved. Did you mean one of the following? [`Any`, `Nombre`, `Codi_Barri`, `Nom_Barri`, `Trimestre`].; line 1 pos 11;
'Aggregate [Nombre#145], [Nombre#145, 'percentile('columna, array(0.25))[0] AS lower_quartile#162, 'percentile('columna, array(0.75))[0] AS upper_quartile#163, 'percentile('columna, array(0.5))[0] AS duration_median#164]
+- Relation [Any#138L,Trimestre#139L,Codi_Districte#140L,Nom_Districte#141,Codi_Barri#142L,Nom_Barri#143,Superfície_mitjana_(m2_construïts)#144,Nombre#145] parquet
